# Vulnerability Ireland

TO DO:
- change census names to 10 characters
- allow the domains and dimensions to be weighted differently

## Script Setup

In [1]:
#load the libraries
library(sf)
library(rgdal)
library(rgeos)
library(raster)
library(tidyverse)
library(RColorBrewer)

Warning message:
"package 'sf' was built under R version 3.6.3"Linking to GEOS 3.9.0, GDAL 3.2.1, PROJ 7.2.1
Loading required package: sp
rgdal: version: 1.4-4, (SVN revision 833)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: C:/Users/jfitton/AppData/Local/Continuum/anaconda3/envs/ISVEHI/Lib/R/library/rgdal/gdal
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: C:/Users/jfitton/AppData/Local/Continuum/anaconda3/envs/ISVEHI/Lib/R/library/rgdal/proj
 Linking to sp version: 1.3-1 
rgeos version: 0.5-1, (SVN revision 614)
 GEOS runtime version: 3.6.1-CAPI-1.10.1 
 Linking to sp version: 1.3-1 
 Polygon checking: TRUE 

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest

In [2]:
#set a directory for export 
exportDirectory <- "../2_OutputData"

# Get the data

In [3]:
#census data
censusData <- read.csv("../1_InputData/1a_CensusData/censusDataZ.csv", header=TRUE, sep=",", stringsAsFactors = FALSE)
# head(censusData)

#impervious surface
imperviousSurfaceData <- read.csv("../1_InputData/1c_ImperviousSurface/imperviousSurface.csv", header=TRUE, sep=",", stringsAsFactors = FALSE)
imperviousSurfaceData <- imperviousSurfaceData %>% select('GUID', 'impervious')
# head(imperviousSurfaceData)

#vulnerabiltiy combine info to help guide the amalgamation of the data - weighting should be changed in this file.
combineInfo <- read.csv("../0_Scripts/VulnerabilityIndicators/vulnerabilityIndicatorsInformation.csv", header=TRUE, sep=",", stringsAsFactors = FALSE, fileEncoding="UTF-8-BOM")
#replace spaces in column names with _
names(combineInfo) <- gsub('\\.', '_', names(combineInfo))
combineInfo


domain,indicator,sens,prepare,respond,recover,enhExp,indVulRel,weighting
age,young,1,0,0,0,0,1,0.5000000
age,old,1,0,0,0,0,1,0.5000000
health,poorHealth,1,0,0,0,0,1,0.5000000
health,disability,1,0,0,0,0,1,0.5000000
income,unemploy,0,1,1,1,0,1,0.2500000
income,lowSkill,0,1,1,1,0,1,0.2500000
income,farming,0,1,1,1,0,1,0.2500000
income,oneParent,0,1,1,1,0,1,0.2500000
info,noInternet,0,1,1,1,0,1,0.3333333
info,education,0,1,1,1,0,1,0.3333333


## Combine the data

In [4]:
#combine the data into one master dataset
masterDataset <- merge(censusData, imperviousSurfaceData, by = 'GUID')
# head(masterDataset)

# Indicator Correlation

Need to check if the indicators are too strongly correlated with each other (+-0.85).

In [5]:
masterDatasetCor <- masterDataset %>% select(-'GUID')
indicatorCorrelation <- round(cor(masterDatasetCor), 2)
indicatorCorrelation

,young,old,priSch,poorHealth,disability,unemploy,lowSkill,farming,rent,education,...,travelTime,noHeating,yearBuilt,mobHome,oneParent,onePerson,noCar,noInternet,priWater,impervious
young,1.00,-0.41,0.65,-0.22,-0.36,-0.15,0.09,-0.07,0.14,-0.16,...,0.23,-0.22,-0.29,-0.01,0.16,-0.35,-0.28,-0.27,-0.03,-0.07
old,-0.41,1.00,-0.39,0.37,0.59,0.12,-0.04,0.11,-0.29,0.21,...,-0.22,0.09,0.26,0.01,-0.15,0.38,0.06,0.39,0.04,-0.10
priSch,0.65,-0.39,1.00,-0.23,-0.34,-0.10,0.14,0.09,-0.11,-0.12,...,0.23,-0.28,-0.29,0.03,0.26,-0.51,-0.45,-0.20,0.12,-0.30
poorHealth,-0.22,0.37,-0.23,1.00,0.68,0.52,0.23,-0.11,0.16,0.34,...,-0.21,0.13,0.11,0.03,0.24,0.35,0.39,0.41,-0.14,0.15
disability,-0.36,0.59,-0.34,0.68,1.00,0.62,0.26,-0.10,0.09,0.47,...,-0.28,0.15,0.16,0.02,0.27,0.45,0.41,0.52,-0.14,0.12
unemploy,-0.15,0.12,-0.10,0.52,0.62,1.00,0.50,-0.04,0.28,0.49,...,-0.25,0.16,0.06,0.07,0.53,0.32,0.46,0.60,-0.09,0.08
lowSkill,0.09,-0.04,0.14,0.23,0.26,0.50,1.00,0.05,0.06,0.23,...,-0.09,-0.01,-0.07,0.00,0.39,0.02,0.10,0.43,0.02,-0.09
farming,-0.07,0.11,0.09,-0.11,-0.10,-0.04,0.05,1.00,-0.47,0.08,...,0.04,0.10,0.27,0.16,-0.20,-0.04,-0.34,0.39,0.72,-0.59
rent,0.14,-0.29,-0.11,0.16,0.09,0.28,0.06,-0.47,1.00,0.03,...,-0.16,0.14,-0.12,-0.12,0.36,0.29,0.66,-0.06,-0.45,0.58
education,-0.16,0.21,-0.12,0.34,0.47,0.49,0.23,0.08,0.03,1.00,...,-0.17,0.08,0.09,0.09,0.18,0.22,0.21,0.43,0.02,-0.07


# Data Processing

## Indicator Relationship

In [6]:
#get the indicator relationship with vulnerability (+1 or -1)
indRelationship <- combineInfo %>% select('indicator', 'indVulRel')
indRelationship <- indRelationship %>%
    spread(key = 'indicator', value = 'indVulRel')
indRelationship


disability,education,engLang,farming,impervious,lowSkill,mobHome,newRes,noCar,noHeating,...,oneParent,onePerson,poorHealth,priSch,priWater,rent,travelTime,unemploy,yearBuilt,young
1,1,1,1,1,1,1,1,1,1,...,1,1,1,-1,1,1,1,1,1,1


In [7]:
#multiply the indicators by their vulnerabiltiy relationship

#get the column names and weights
names <- names(indRelationship)
weights <- indRelationship[, names]

#copy and rename the dataset
indicatorRelationshipDataset <- masterDataset

#multiply the data
indicatorRelationshipDataset[, names] <- sweep(indicatorRelationshipDataset[, names], 2, unlist(weights[, names]), "*")
# indicatorRelationshipDataset


## Domains

In [8]:
# get the domains and their associated indicator ID
domainsIndicators <- combineInfo %>% select('domain', 'indicator')
domainsIndicators

domain,indicator
age,young
age,old
health,poorHealth
health,disability
income,unemploy
income,lowSkill
income,farming
income,oneParent
info,noInternet
info,education


In [9]:
#get a list of the unique domains
domains <- unique(domainsIndicators$domain)
domains

[1] "age"      "health"   "income"   "info"     "locKnow"  "mobility"
 [7] "access"   "socNet"   "tenure"   "physEnv"  "houChar"

In [10]:
#create the start of the dataset in which further data will be added with the GUID
domainDatasetMerge <- masterDataset %>% select('GUID')

#loop through each the domains and:
for (domainI in domains){
    #identify which indicators are used within the domain
    domainData <- domainsIndicators %>% filter(domain == domainI)
    
    #get a vector of indicator IDs for the domain
    domainDataIDs <- domainData$indicator
#     print(domainDataIDs)
  
    #count the number of indicators in the domain
    indicatorCount <- length(domainData$indicator)
#     print(indicatorCount)
    
    #add the GUID column name to the vector
    domainDataIDs <- (c('GUID', domainDataIDs))
#     str(domainDataIDs)
    
#     filter the master dataset to only the indicators in the domain
    masterDatasetDomainFiltered <- indicatorRelationshipDataset[domainDataIDs]
#    str(masterDatasetDomainFiltered)

    #calculate the weight of each indicator (assumed to be EQUAL)
    weighting <- 1/indicatorCount
#     print(weighting)
          
    #weight the data
    weightedData <- masterDatasetDomainFiltered %>% mutate_if(is.numeric, function(x) {x*weighting})
#     str(weightedData)
    
    
    #sum the data to get the total score for the domain
    weightedData[, domainI] <- rowSums(weightedData[2:(indicatorCount+1)], na.rm = TRUE)
    
    #add the total for the domain to the merging table
    domainDatasetMerge <- merge(domainDatasetMerge, weightedData %>% select('GUID', domainI), by = 'GUID')
    
}

#all the total domain scores collated into one table
head(domainDatasetMerge)


GUID,age,health,income,info,locKnow,mobility,access,socNet,tenure,physEnv,houChar
4c07d11d-f3af-851d-e053-ca3ca8c0ca7f,-0.27571070,-0.3550708,0.4705805,0.2433894,-0.4063267,-0.3767343,-0.06620701,-0.1978261,-0.9981907,-0.9943521,1.0986106
4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f,-0.61652655,0.2368063,0.2512133,0.1872005,0.0791042,-0.8770271,0.52329204,0.2392538,-1.1366477,-0.9821737,0.7909747
4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f,-0.05791998,-0.7135048,0.1110740,-0.0759769,-0.6806991,-0.8872241,0.57102315,-1.1227001,-1.1537786,-0.9924398,0.2656963
4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f,0.08915214,-0.7169866,0.2510038,-0.2558190,-0.9153823,-0.5618455,1.06598490,-0.3383713,-1.0934459,-0.9866611,0.4395729
4c07d11d-f3b3-851d-e053-ca3ca8c0ca7f,-0.01209118,-0.7994567,0.3193815,0.2053121,-0.4149132,-0.8611234,0.61818818,-0.8218861,-0.8361234,-0.9561675,0.5269111
4c07d11d-f3b4-851d-e053-ca3ca8c0ca7f,-0.19932908,-0.0747851,0.4565445,0.1642002,-0.9153823,-0.6656248,1.22650737,-0.0111402,-1.1276444,-0.9747931,0.7954311


## Dimensions

Need to collate the domains into the five dimensions.

In [11]:
# create a vector of the dimension names
dimensions <- c('sens', 'prepare', 'respond', 'recover', 'enhExp')

#create the dataset to combine the results with just the GUID present
dimensionDatasetMerge <- masterDataset %>% select('GUID')

# get the domain and dimension information
dimensionInfo <- combineInfo %>% select(c('domain', dimensions))
# dimensionInfo

# loop through each of the dimensions and:
for (dimensionI in dimensions){
    #get all of the domains and a single dimension
    dimensionIndicators <- dimensionInfo %>% select(c('domain', dimensionI))
    
    #filter the domains by the dimension - 1 means that the domain is part of the dimension
    dimensionIndicators <- dimensionIndicators %>% filter(dimensionIndicators[, dimensionI] == 1)
       
    #get a unique vector of the domains
    domainsDimensionIndicator <- unique(dimensionIndicators$domain)
    
    #get the number of domains that is in the dimension
    domainsDimensionIndicatorCount = length(domainsDimensionIndicator)
#     print(domainsDimensionIndicatorCount)
    
    #create a dataset with only the domains for the dimension
    dimensionTotal <- domainDatasetMerge %>% select(c('GUID', domainsDimensionIndicator))  
    str(dimensionTotal)
    
#     #create the new dimension field name
#     dimensionName = paste(dimensionI)
    
    #get the sum of the dimensions
    dimensionTotal[, dimensionI] <- rowSums(dimensionTotal[2:(domainsDimensionIndicatorCount+1)], na.rm = TRUE)
#      str(dimensionTotal)
    
    #add the total for the dimension to the merging table
    dimensionDatasetMerge <- merge(dimensionDatasetMerge, dimensionTotal %>% select('GUID', dimensionI), by = 'GUID')   
    
}
head(dimensionDatasetMerge)



'data.frame':	18641 obs. of  3 variables:
 $ GUID  : chr  "4c07d11d-f3af-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f" ...
 $ age   : num  -0.2757 -0.6165 -0.0579 0.0892 -0.0121 ...
 $ health: num  -0.355 0.237 -0.714 -0.717 -0.799 ...
'data.frame':	18641 obs. of  5 variables:
 $ GUID   : chr  "4c07d11d-f3af-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f" ...
 $ income : num  0.471 0.251 0.111 0.251 0.319 ...
 $ info   : num  0.243 0.187 -0.076 -0.256 0.205 ...
 $ locKnow: num  -0.4063 0.0791 -0.6807 -0.9154 -0.4149 ...
 $ tenure : num  -0.998 -1.137 -1.154 -1.093 -0.836 ...
'data.frame':	18641 obs. of  7 variables:
 $ GUID    : chr  "4c07d11d-f3af-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f" ...
 $ income 

GUID,sens,prepare,respond,recover,enhExp
4c07d11d-f3af-851d-e053-ca3ca8c0ca7f,-0.6307815,-0.6905475,-0.3331242,0.13940956,0.1042585
4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f,-0.3797203,-0.6191297,0.4030368,-0.19935948,-0.1911991
4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f,-0.7714247,-1.7993806,-2.0845031,-1.97482716,-0.7267435
4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f,-0.6278345,-2.0136435,-0.7544294,-0.90503197,-0.5470883
4c07d11d-f3b3-851d-e053-ca3ca8c0ca7f,-0.8115478,-0.7263430,-0.9550410,-1.15831600,-0.4292565
4c07d11d-f3b4-851d-e053-ca3ca8c0ca7f,-0.2741142,-1.4222821,0.2551047,-0.05602034,-0.1793620


In [12]:
#standardise the data

#copy the data so that Z scores can be calculated
dimensionDatasetMergeZ <- dimensionDatasetMerge

#if the column is a numeric, then scale the data
dimensionDatasetMergeZ %>% mutate_if(is.numeric, scale)

head(dimensionDatasetMergeZ)

GUID,sens,prepare,respond,recover,enhExp
4c07d11d-f3af-851d-e053-ca3ca8c0ca7f,-0.54967156,-0.3398043,-0.1320824,0.06160496,0.10039185
4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f,-0.33089341,-0.3046611,0.1598024,-0.08809677,-0.18410802
4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f,-0.67223001,-0.8854384,-0.8264970,-0.87267434,-0.69979059
4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f,-0.54710350,-0.9908728,-0.2991282,-0.39993281,-0.52679827
4c07d11d-f3b3-851d-e053-ca3ca8c0ca7f,-0.70719383,-0.3574185,-0.3786699,-0.51185880,-0.41333651
4c07d11d-f3b4-851d-e053-ca3ca8c0ca7f,-0.23886682,-0.6998759,0.1011480,-0.02475534,-0.17270997
4c07d11d-f3b5-851d-e053-ca3ca8c0ca7f,-0.63975770,-0.6314529,-0.6447030,-0.91492765,-0.17094418
4c07d11d-f3b6-851d-e053-ca3ca8c0ca7f,-0.34134843,-0.5431067,-0.2489939,-0.19896973,-0.46643999
4c07d11d-f3b7-851d-e053-ca3ca8c0ca7f,-0.33315475,-0.3781402,-0.1555424,-0.52297593,-0.71471064
4c07d11d-f3b8-851d-e053-ca3ca8c0ca7f,0.36021329,-0.5925083,0.1017963,-0.10475633,-0.18957729


GUID,sens,prepare,respond,recover,enhExp
4c07d11d-f3af-851d-e053-ca3ca8c0ca7f,-0.6307815,-0.6905475,-0.3331242,0.13940956,0.1042585
4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f,-0.3797203,-0.6191297,0.4030368,-0.19935948,-0.1911991
4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f,-0.7714247,-1.7993806,-2.0845031,-1.97482716,-0.7267435
4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f,-0.6278345,-2.0136435,-0.7544294,-0.90503197,-0.5470883
4c07d11d-f3b3-851d-e053-ca3ca8c0ca7f,-0.8115478,-0.7263430,-0.9550410,-1.15831600,-0.4292565
4c07d11d-f3b4-851d-e053-ca3ca8c0ca7f,-0.2741142,-1.4222821,0.2551047,-0.05602034,-0.1793620


## Vulnerability

In [13]:
#sum the dimensions to create a total overall score of vulnerability
dimensionDatasetMergeZ$vuln <- rowSums(dimensionDatasetMergeZ[2:(ncol(dimensionDatasetMergeZ))], na.rm = TRUE)
head(dimensionDatasetMergeZ)

#scale the overall vulnerability
dimensionDatasetMergeZ$vuln <- scale(dimensionDatasetMergeZ$vuln)

#scale makes the data a list(?), so need to make it numeric to allow the shapefile to be created at export
dimensionDatasetMergeZ$vuln<- as.numeric(dimensionDatasetMergeZ$vuln)
head(dimensionDatasetMergeZ)

GUID,sens,prepare,respond,recover,enhExp,vuln
4c07d11d-f3af-851d-e053-ca3ca8c0ca7f,-0.6307815,-0.6905475,-0.3331242,0.13940956,0.1042585,-1.4107851
4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f,-0.3797203,-0.6191297,0.4030368,-0.19935948,-0.1911991,-0.9863717
4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f,-0.7714247,-1.7993806,-2.0845031,-1.97482716,-0.7267435,-7.3568791
4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f,-0.6278345,-2.0136435,-0.7544294,-0.90503197,-0.5470883,-4.8480276
4c07d11d-f3b3-851d-e053-ca3ca8c0ca7f,-0.8115478,-0.7263430,-0.9550410,-1.15831600,-0.4292565,-4.0805043
4c07d11d-f3b4-851d-e053-ca3ca8c0ca7f,-0.2741142,-1.4222821,0.2551047,-0.05602034,-0.1793620,-1.6766739


GUID,sens,prepare,respond,recover,enhExp,vuln
4c07d11d-f3af-851d-e053-ca3ca8c0ca7f,-0.6307815,-0.6905475,-0.3331242,0.13940956,0.1042585,-0.193199
4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f,-0.3797203,-0.6191297,0.4030368,-0.19935948,-0.1911991,-0.135078
4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f,-0.7714247,-1.7993806,-2.0845031,-1.97482716,-0.7267435,-1.007483
4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f,-0.6278345,-2.0136435,-0.7544294,-0.90503197,-0.5470883,-0.663910
4c07d11d-f3b3-851d-e053-ca3ca8c0ca7f,-0.8115478,-0.7263430,-0.9550410,-1.15831600,-0.4292565,-0.558802
4c07d11d-f3b4-851d-e053-ca3ca8c0ca7f,-0.2741142,-1.4222821,0.2551047,-0.05602034,-0.1793620,-0.229611


## Merge

In [14]:
#merge all the indicators, domains, dimensions, and total vulnerability into one dataset
outputDataset <- merge(masterDataset, domainDatasetMerge, by = 'GUID')
outputDataset <- merge(outputDataset, dimensionDatasetMergeZ, by = 'GUID')
str(outputDataset)

'data.frame':	18641 obs. of  40 variables:
 $ GUID      : chr  "4c07d11d-f3af-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f" "4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f" ...
 $ young     : num  -0.6249 -1.1646 -0.0442 0.3839 0.3004 ...
 $ old       : num  0.0735 -0.0685 -0.0717 -0.2056 -0.3246 ...
 $ priSch    : num  0.00768 -0.64727 1.36085 0.72923 0.79468 ...
 $ poorHealth: num  -0.615 0.565 -0.539 -0.815 -0.704 ...
 $ disability: num  -0.0953 -0.0912 -0.8877 -0.6192 -0.8945 ...
 $ unemploy  : num  -0.1812 -0.1218 -0.789 -0.0739 -0.4991 ...
 $ lowSkill  : num  -0.1146 -0.0376 -1.5783 1.4517 -0.0587 ...
 $ farming   : num  1.87 1.56 3.23 0.35 1.9 ...
 $ rent      : num  -0.998 -1.137 -1.154 -1.093 -0.836 ...
 $ education : num  0.479 -0.223 0.379 0.116 0.248 ...
 $ engLang   : num  -0.557 -0.696 -0.696 -0.696 -0.469 ...
 $ newRes    : num  -0.4063 0.0791 -0.6807 -0.9154 -0.4149 ...
 $ travelTime: num  -0.0662 0.5233 0.571 1.066 0.6182 .

# Export

In [24]:
#set the export path
exportPath <- paste(exportDirectory, 'irelandVulnerabilty.csv', sep = '/')

#write the output to a csv
write.csv(outputDataset, exportPath, row.names = FALSE)

#get the small area spatial data
smallAreaShp <- st_read("../0_Scripts/downloads/d9876b9b-96eb-4f04-afec-300c5d0a276c2020328-1-wq0rek.ua66c.shp")

#get just the GUID from the dataset
smallAreaShp <- smallAreaShp[c('GUID')]

# #add the data to the spatial data using the GUID as the common field
smallAreaVulnerability <- merge(smallAreaShp, outputDataset, by ="GUID")

#export to shp and geojson
st_write(smallAreaVulnerability, paste(exportDirectory, "/", "irelandVulnerability.geojson", sep = ""))
st_write(smallAreaVulnerability, paste(exportDirectory, "/", "irelandVulnerability.shp", sep = ""))

Writing layer `irelandVulnerability' to data source `../2_OutputData/irelandVulnerability.shp' using driver `ESRI Shapefile'
Writing 18641 features with 40 fields and geometry type Multi Polygon.


**END**